In [ ]:
#Date
response.xpath('//span[@class="post-timestamp"]/text()').extract()
    
#Buff/nerf/tweaks
response.xpath('//div[@class="post-body entry-content"]//b//text()').extract()

#Changed Champs
#response.xpath('//div[@class="post-body entry-content"]/text()').extract()
#response.xpath('//div[@class="post-body entry-content"]/div[2]//text()').extract()
response.xpath('//div[@class="post-body entry-content"]//b//following-sibling::text()').extract()

In [ ]:
starturl = 'www.nerfplz.com/search/label/Patch Notes?max-results=50'
#patch links
response.xpath('//span[@class="rmlink"]/a[contains(text(), "Read More...")]/@href').extract()

#next page
response.xpath('//span[@id="blog-pager-older-link"]/a/@href').extract()